Today's Topic
--------------
→ case when in spark SQL.

→ when otherwise in Pyspark.

→ How to deal with null value?

→ case when | when otherwise with multiple AND,OR condition.

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

#Defining schema
schema = ['id','name','age','salary','country','dept']

#Creating employee dataframe
emp_df = spark.createDataFrame(data=emp_data,schema = schema)

In [0]:
# Show emp_df
emp_df.show(truncate=False)

+----+-------+----+------+-------+-----------+
|id  |name   |age |salary|country|dept       |
+----+-------+----+------+-------+-----------+
|1   |manish |26  |20000 |india  |IT         |
|2   |rahul  |null|40000 |germany|engineering|
|3   |pawan  |12  |60000 |india  |sales      |
|4   |roshini|44  |null  |uk     |engineering|
|5   |raushan|35  |70000 |india  |sales      |
|6   |null   |29  |200000|uk     |IT         |
|7   |adam   |37  |65000 |us     |IT         |
|8   |chris  |16  |40000 |us     |sales      |
|null|null   |null|null  |null   |null       |
|7   |adam   |37  |65000 |us     |IT         |
+----+-------+----+------+-------+-----------+



In [0]:
# Show the employees who are adult and who are not by creating new column named "adult"
from pyspark.sql.functions import *

emp_df.withColumn("adult",when(emp_df.age > 18,"Yes")
                         .when(col("age") < 18,"No")
                         .otherwise("No Value")
                 ).show(truncate=False)

+----+-------+----+------+-------+-----------+--------+
|id  |name   |age |salary|country|dept       |adult   |
+----+-------+----+------+-------+-----------+--------+
|1   |manish |26  |20000 |india  |IT         |Yes     |
|2   |rahul  |null|40000 |germany|engineering|No Value|
|3   |pawan  |12  |60000 |india  |sales      |No      |
|4   |roshini|44  |null  |uk     |engineering|Yes     |
|5   |raushan|35  |70000 |india  |sales      |Yes     |
|6   |null   |29  |200000|uk     |IT         |Yes     |
|7   |adam   |37  |65000 |us     |IT         |Yes     |
|8   |chris  |16  |40000 |us     |sales      |No      |
|null|null   |null|null  |null   |null       |No Value|
|7   |adam   |37  |65000 |us     |IT         |Yes     |
+----+-------+----+------+-------+-----------+--------+



In [0]:
# Let's consider another scenario in which we have asked to fix the age if there is any null value then go for 
# identifying which employees are adult and who are not
emp_df.withColumn("age",when(emp_df["age"].isNull(),lit(19))
                       .otherwise(col("age")))\
      .withColumn("adult",when(col("age") > 18,"Yes")
                         .otherwise("No")).show()

+----+-------+---+------+-------+-----------+-----+
|  id|   name|age|salary|country|       dept|adult|
+----+-------+---+------+-------+-----------+-----+
|   1| manish| 26| 20000|  india|         IT|  Yes|
|   2|  rahul| 19| 40000|germany|engineering|  Yes|
|   3|  pawan| 12| 60000|  india|      sales|   No|
|   4|roshini| 44|  null|     uk|engineering|  Yes|
|   5|raushan| 35| 70000|  india|      sales|  Yes|
|   6|   null| 29|200000|     uk|         IT|  Yes|
|   7|   adam| 37| 65000|     us|         IT|  Yes|
|   8|  chris| 16| 40000|     us|      sales|   No|
|null|   null| 19|  null|   null|       null|  Yes|
|   7|   adam| 37| 65000|     us|         IT|  Yes|
+----+-------+---+------+-------+-----------+-----+



In [0]:
emp_df.withColumn("age_category",when((col("age") > 0 ) & (col("age") <= 18),"Minor")
                            .when((col("age") > 18 ) & (col("age") <= 30),"Med")
                            .when((col("age") > 30 ) & (col("age") <= 100),"Major")
                            .otherwise("No Value")\
                 ).show()


+----+-------+----+------+-------+-----------+------------+
|  id|   name| age|salary|country|       dept|age_category|
+----+-------+----+------+-------+-----------+------------+
|   1| manish|  26| 20000|  india|         IT|         Med|
|   2|  rahul|null| 40000|germany|engineering|    No Value|
|   3|  pawan|  12| 60000|  india|      sales|       Minor|
|   4|roshini|  44|  null|     uk|engineering|       Major|
|   5|raushan|  35| 70000|  india|      sales|       Major|
|   6|   null|  29|200000|     uk|         IT|         Med|
|   7|   adam|  37| 65000|     us|         IT|       Major|
|   8|  chris|  16| 40000|     us|      sales|       Minor|
|null|   null|null|  null|   null|       null|    No Value|
|   7|   adam|  37| 65000|     us|         IT|       Major|
+----+-------+----+------+-------+-----------+------------+



In [0]:
# Let's do the same thing in SparkSQL using case when
# At first we need to convert the dataframe into temporary view
emp_df.createOrReplaceTempView("employee_tbl")

In [0]:
spark.sql("""
          SELECT *,
            CASE WHEN age > 18 THEN "Yes" 
                 WHEN age < 18 THEN "No"
                 ELSE "No Value"
            END AS Adult
          FROM employee_tbl
          """).show()

+----+-------+----+------+-------+-----------+--------+
|  id|   name| age|salary|country|       dept|   Adult|
+----+-------+----+------+-------+-----------+--------+
|   1| manish|  26| 20000|  india|         IT|     Yes|
|   2|  rahul|null| 40000|germany|engineering|No Value|
|   3|  pawan|  12| 60000|  india|      sales|      No|
|   4|roshini|  44|  null|     uk|engineering|     Yes|
|   5|raushan|  35| 70000|  india|      sales|     Yes|
|   6|   null|  29|200000|     uk|         IT|     Yes|
|   7|   adam|  37| 65000|     us|         IT|     Yes|
|   8|  chris|  16| 40000|     us|      sales|      No|
|null|   null|null|  null|   null|       null|No Value|
|   7|   adam|  37| 65000|     us|         IT|     Yes|
+----+-------+----+------+-------+-----------+--------+



In [0]:
# Let's fix the null value and then identifying whose employees are adult and who are not
spark.sql("""
        SELECT id,name,updated_age,salary,country,dept,Adult
        FROM(
         SELECT *,
            CASE WHEN age IS NULL THEN 19 ELSE age END AS updated_age,
            CASE WHEN updated_age > 18 THEN "Yes" ELSE "NO" END AS Adult
          FROM employee_tbl
        )
        """).show(truncate=False)
# We can also do it in another way like at first we need to update the table then 
# We can apply case when statement for our desired result

+----+-------+-----------+------+-------+-----------+-----+
|id  |name   |updated_age|salary|country|dept       |Adult|
+----+-------+-----------+------+-------+-----------+-----+
|1   |manish |26         |20000 |india  |IT         |Yes  |
|2   |rahul  |19         |40000 |germany|engineering|Yes  |
|3   |pawan  |12         |60000 |india  |sales      |NO   |
|4   |roshini|44         |null  |uk     |engineering|Yes  |
|5   |raushan|35         |70000 |india  |sales      |Yes  |
|6   |null   |29         |200000|uk     |IT         |Yes  |
|7   |adam   |37         |65000 |us     |IT         |Yes  |
|8   |chris  |16         |40000 |us     |sales      |NO   |
|null|null   |19         |null  |null   |null       |Yes  |
|7   |adam   |37         |65000 |us     |IT         |Yes  |
+----+-------+-----------+------+-------+-----------+-----+

